# Cleaning and Combining Victoria 2013-2023 Residential Data
In this notebook we clean and combine Victoria Government 2013-2023 house, unit and land data into a single dataframe. This will be used to create an excel dashboard.

In [8]:
# import modules
import pandas as pd

In [9]:
# read in data
house_prices = pd.read_excel("./raw/Houses-by-suburb-2013-2023.xlsx")
unit_prices = pd.read_excel("./raw/Units-by-suburb-2013-2023.xlsx")
land_prices = pd.read_excel("./raw/Vacant-land-by-suburb-2013-2023.xlsx")

prices = [house_prices, unit_prices, land_prices]
prices[0].head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Change,Unnamed: 13,Unnamed: 14,Growth
0,Locality,2013,2014,2015,2016,2017,2018,2019,2020.0,2021.0,2022.0,2023.0,Prelim,22-23,13-23,PA
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,(%),(%),13-23
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(%)
3,ABBOTSFORD,792500,862500,925000,1187500,1280000,1192500,1050000,1200000.0,1365000.0,1346000.0,1250000.0,1027500,-7,58,4.7
4,ABERFELDIE,947500,1045000,1207500,1300000,1471000,1498500,1390000,1520000.0,1858000.0,1900000.0,1631000.0,3900000,-14,72,5.6


##### Align Column and Type Formatting and Impute Missing Values

In [10]:
# align column formatting
col_names = house_prices.iloc[0]
col_names = [int(x) if isinstance(x, float) else x for x in col_names]

for type in prices:

    # align column formatting 
    type.columns = col_names
    type.drop(index= [0, 1, 2], axis = 0, inplace=True)
    type.drop(columns= ['Prelim      ', '   22-23', '13-23', 'PA'], axis = 1, inplace=True)
    type.reset_index(drop=True, inplace=True)


    # type formatting 
    for i in range(len(type.columns[1:])):
        year = type.columns[1:][i]

        # record values to be imputed for this column
        sum = 0
        missing = []
        
        for j in range(len(type[year])):
            
            # if float cast to int
            if isinstance(type.iloc[j][year], float):
                type.at[j, year] = int(type.iloc[j][year])
                sum += type.iloc[j][year]
                continue
                
            # if missing value then imput using previous years price
            if type.iloc[j][year] == '-':
                
                # if last years value is available then use it
                if i > 0:
                    type.at[j, year] = type.at[j, year - 1]
                    sum += type.iloc[j][year]

                # otherwise 
                else: 
                    missing.append(j)
                
                continue
            
            if isinstance(type.iloc[j][year], str) and type.at[j, year][-1] == '*':
                type.at[j, year] = int(type.at[j, year][:-1])
                sum += type.iloc[j][year]
                continue 
            
            type.at[j, year] = int(type.at[j, year])

            sum += type.iloc[j][year]

        # imput still missing values as average of the column 
        if missing: 
            avg = int (sum / len(type[year]))
            for j in missing:
                type.at[j, year] = avg

        # set column data type
        type[year] = type[year].astype('int64')

    # removing spaces form Locality 
    type['Locality'] = [x.strip() for x in type['Locality']]

prices[0].head()    
                

,Locality,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,ABBOTSFORD,792500,862500,925000,1187500,1280000,1192500,1050000,1200000,1365000,1346000,1250000
1,ABERFELDIE,947500,1045000,1207500,1300000,1471000,1498500,1390000,1520000,1858000,1900000,1631000
2,AINTREE,470407,470407,600000,600000,571000,557500,575000,616000,713500,756000,748000
3,AIREYS INLET,664000,625500,680000,715000,737500,869000,985000,1132500,1775000,1725000,1675000
4,AIRPORT WEST,532000,575000,635000,742000,845000,845000,795000,812500,960000,911000,945000


##### Create Single New Dataframe

In [11]:

cols = ['Suburb', 'Residential Type', 'Year', 'Price']
residential_types = ["House", "Unit", "Land"]

# values for columns
Suburb_vals = []
Type_vals = []
Year_vals = []
Price_vals = []
data = [Suburb_vals, Type_vals, Year_vals, Price_vals]

# capture data 
for i in range(len(prices)):
    type = residential_types[i]
    source = prices[i]
    n_rows = source.shape[0]
    n_years = source.shape[1] - 1
    start_year = source.columns[1]

    # every row turns into number of years rows 
    for j in range(n_rows * n_years):
        row = j // n_years
        year = start_year + j % n_years 

        Suburb_vals.append(source.iloc[row]['Locality'])
        Type_vals.append(type)
        Year_vals.append(year)
        Price_vals.append(source.iloc[row][year])

# create dataframe
residential = pd.DataFrame({cols[i]: data[i] for i in range(len(cols))})
residential.head()

,Suburb,Residential Type,Year,Price
0,ABBOTSFORD,House,2013,792500
1,ABBOTSFORD,House,2014,862500
2,ABBOTSFORD,House,2015,925000
3,ABBOTSFORD,House,2016,1187500
4,ABBOTSFORD,House,2017,1280000


In [14]:
residential.shape

(15873, 4)

In [12]:
# save to excel format
residential.to_excel("./processed/Median Residential Prices.xlsx", index=False)